# Trefethen Problem 2
February 25, 2023

> A photon moving at speed 1 in the x-y plane starts at t = 0 at (x, y) = (1/2, 1/10)
heading due east. Around every integer lattice point (i, j) in the plane, a circular
mirror of radius 1/3 has been erected. How far from the origin is the photon at
t = 10?


## Notes:

1. The problem has chosen units so that the speed of light is 1.
1. Clearly if the photon is completely between the rows or columns of mirrors and moving parallel to them, the photon will go on forever without hitting any mirror. However it cannot get to any of these paths by bouncing off of a mirror.
1. What happens if the photon happens to be travelling so that it becomes exactly tangent to one of the mirrors? If you think about this in the limit as the angle of the incident ray changes, clearly the ray would continue straight as if it didn't hit the mirror.


## Finding collision points

Finding where the photon collides with a mirror is equivalent to finding the points of intersection between a line and a circle. We can start by writing the slope-intercept equation for a line
$$
y = mx+b.
$$

The equation of a circle with center at $x=h, y=k$ and radius $r$ is given by

$$
(x-h)^2 + (y-k)^2 = r^2.
$$

We can substitue the value of $y$ from the line equation into the circle equation to get an equation with only $x$ as the unknown, namely

$$
(mx-k+b)^2 + (x-h)^2 = r^2
$$

which can be solved for $x$:

$$
x=\frac{\pm\sqrt{\left( {{m}^{2}}+1\right) \, {{r}^{2}}-{{h}^{2}}\, {{m}^{2}}+\left( 2 h k-2 b h\right)  m-{{k}^{2}}+2 b k-{{b}^{2}}}+\left( k-b\right)  m+h}{{{m}^{2}}+1}
$$

However, the line for the photon's trajectory is not given in slope-intercept form. In practice we know the position of the photon $(p_x, p_y)$ and a 2D vector for its velocity $\vec{v} = v_x \hat{i} + v_y \hat{j}$. From this information we can find the slope $m$ and intercept $b$ from
$$
\begin{align}
m &= \frac{v_y}{v_x} \\
b &= p_y - \frac{p_x v_y}{v_x}
\end{align}
$$
and substitute these values into the solution above.

There is still the problem of how we handle the situation where $v_x=0$, since we then have an infinite slope with a vertical trajectory and the solution equation blows up. However in this case the equation of the line is just

$$
x = p_x
$$

and we can substitute this into the circle equation to get a solution for $y$:

$$
y=k \pm \sqrt{r^2 - p_x^{2} +2h p_x - h^2}.
$$

In [1]:
function intersections(center::Vector{Int64}, r::BigFloat, p::Vector{BigFloat}, v::Vector{BigFloat})
    #=
    Finds the intersection points of a line with a circle, with the line
    specified by a vector v originating from a point p.
    
    Arguments:
        center: 2D vector for the center of the circle
        r: radius of the circle
        p: 2D vector for the position of the photon
        v: 2D vector for the velocity of the photon

    Returns: a 2-tuple
        If there are 2 intersection points, they are returned as
            (Vector{BigFloat}, Vector{BigFloat})
        If there is just one intersection point it is returned as
        the first tuple value, with nothing in the second value
            (Vector{BigFloat}, nothing)
        If there are no intersection points this function returns
        both tuple values as nothing
            (nothing, nothing)
    =#
    ret = (nothing, nothing)
    h = BigFloat(center[1])
    k = BigFloat(center[2])
    
    # Check for a vertical line
    if v[1] == 0
        # The trajectory is vertical
        disc = r^2 - p[1]^2 + 2h*p[1] - h^2
        if disc == 0
            ret = ([p[1],k], nothing)
        elseif disc > 0
            sq = sqrt(disc)
            ret = ([p[1],k+sq], [p[1],k-sq])
        end
    else
        # The trajectory is not vertical
        m = v[2] / v[1]
        b = p[2] - m*p[1]
        
        disc = (m^2 + 1)*r^2 - (h*m)^2 + 2h*m*(k-b) - k^2 + 2b*k - b^2
        if disc == 0
            #println("One solution")
            x1 = ((k - b)*m + h) / (m^2 + 1)
            y1 = m*x1 + b
            ret = ([x1,y1], nothing)
        elseif disc > 0
            #println("Two solutions")
            sq = sqrt(disc)
            x1 = ((k - b)*m + h + sq) / (m^2 + 1)
            y1 = m*x1 + b
            x2 = ((k - b)*m + h - sq) / (m^2 + 1)
            y2 = m*x2 + b
            ret = ([x1,y1], [x2,y2])
        end
    end
    
    return ret
end

intersections (generic function with 1 method)

The function below determines whether a photon in a given position is traveling in a direction such that it will collide with a circle. Using the function **intersections** above it can be determined whether the line the photon is following intersects the circle. However, if that line does intersect the circle, we still have two cases to consider:
1. The photon is traveling away from the intersection point, so it never collides with the circle.
1. The photon is traveling towards the intersection point, and so *will* collide with the circle.


In [2]:
function photon_intersect(center::Vector{Int64}, r::BigFloat, p::Vector{BigFloat}, v::Vector{BigFloat})
    #=
    Determines the collision point between a photon and a circular mirror, if any.

    Arguments:
        center: 2D vector for the center of the circle
        r: radius of the circle
        p: 2D vector for the position of the photon
        v: 2D vector for the velocity of the photon

    Returns: nothing if there is no collision point, else
             returns the coordinates of the collision point
    =#
    bigzero = BigFloat("0")
    retval = nothing
    coll1, coll2 = intersections(center, r, p, v)
    if coll1 == nothing
        return nothing  # No collision point
    elseif coll2 == nothing
        # Form the vector from the photon position to the collision point
        pc = coll1 .- p

        # Take the inner product with the velocity vector
        inner_prod = sum(pc .* v)

        if inner_prod > 0.0
            retval = coll1
        end
    else
        # Two intersection points - determine if one or both will
        # be reached by the photon
        pc1 = coll1 .- p
        inner_prod1 = sum(pc1 .* v)
        reached1 = inner_prod1 > bigzero
        
        pc2 = coll2 .- p
        inner_prod2 = sum(pc2 .* v)
        reached2 = inner_prod2 > bigzero
        
        if reached1
            if reached2
                # Both collision points are reached. Find which is closest.
                if inner_prod1 < inner_prod2
                    retval = coll1
                else
                    retval = coll2
                end
            else
                retval = coll1
            end
        elseif reached2
            retval = coll2
        end
    end

    return retval
end

photon_intersect (generic function with 1 method)

## Calculating the new velocity vector

From http://www.paulbourke.net/geometry/reflected/ the formula for the reflected velocity vector is

$$
{\bf v}_r = {\bf v}_i - 2 {\bf N} ({\bf v}_i \cdot {\bf N})
$$
where
- ${\bf v}_r$ is the reflected velocity vector
- ${\bf v}_i$ is the incident velocity vector
- ${\bf N}$ is the normal vector at the collision point

In [3]:
function new_velocity(center::Vector{Int64}, r::BigFloat, v_i::Vector{BigFloat}, coll::Vector{BigFloat})
    #=
    Determine the new velocity of a photon after a collision.
    
    Arguments:
        center: 2D vector for the center of the circle
        r: radius of the circle
        v_i: 2D vector for the incident velocity of the photon
        coll: 2D vector for the collision point
    =#

    # Calculate the unit normal at the collision point
    fcenter = (BigFloat.(center))
    nv = (coll .- fcenter)
    N = nv ./ sqrt(sum(nv.^2))
    #println("N = $N")

    # Compute the component of the incident velocity along the normal
    vi_N = sum(v_i .* N)

    # Compute the reflected velocity
    v_r = v_i .- (BigFloat("2") * vi_N) .* N
    return v_r
end

new_velocity (generic function with 1 method)

A quick sketch of the problem reveals that the first collision will occur with the mirror centered at $(1,0)$. From that point forward the problem is to determine the next circle the ray will collide with. The geometry of the problem is such that the ray cannot strike the same mirror immediately because the surface is convex. We will have the components of the velocity vector after the collision, which will allow us to limit the search to circles in a quadrant around the current circle. For example, if the velocity following the first collision had a positive $y$ component and a negative $x$ component, can immediately say that the second collision will be with a circle having a center $(x_2, y_2)$ where $x_2 \le 1$ and $y_2 \ge 0$.

Actually we can go even further than that. From the $(1,0)$ cell, the only cells that can be reached by the ray are
- Adjacent cells $(0,1)$, $(0,0)$, $(1,1)$
- $(0,j)$ for $j\ge 1$
- $(i,1)$ for $i\le -1$.

Additionally we can look at the relative sizes of the $x$ and $y$ components of the velocity to limit the choices even more. For example, if $v_x > 0, v_y > 0$, and $v_x < v_y$, the only cells possible from cell $(i,j)$ are
- Adjacent cell just above at $(i,j+1)$
- Cells $(i+1,k)$ for $k\gt j$.


In [4]:
function run(final_t::BigFloat)
    # Set numerical precision
    setprecision(BigFloat, 256)

    # Initialize the trajectory
    total_t = BigFloat("0")
    center = [1,0]
    r = BigFloat("1.0")/BigFloat("3")
    p = [BigFloat("0.5"), BigFloat("0.1")]
    v = [BigFloat("1"), BigFloat("0")]

    # Find first collision point and new velocity
    coll = photon_intersect(center, r, p, v)
    v = new_velocity(center, r, v, coll)

    # Compute total time (distance) taken
    displacement = coll .- p
    total_t += sqrt(sum(displacement.^2))
    center = [1,0]

    cell = copy(center)
    done = false
    while !done
        # center - holds the cell where collision occurred
        # cell - holds cell we are testing for next collision
        center = copy(cell)
        p = coll

        # Determine up/down and left/right
        if v[2] > 0.0
            Δy = 1   # Going up
        else
            Δy = -1  # Going down
        end

        if v[1] >= 0.0
            Δx = 1   # Going right
        else
            Δx = -1  # Going left
        end

        # Determine if slope of v is more vertical or horizontal
        vertical = (abs(v[1]) <= abs(v[2]))

        # Check adjacent cell for collision
        if vertical
            cell = [center[1], center[2]+Δy]
        else
            cell = [center[1]+Δx, center[2]]
        end
        coll = photon_intersect(cell, r, p, v)

        if coll == nothing
            # Check corner cell
            cx = center[1] + Δx
            cy = center[2] + Δy
            cell = [cx,cy]
            coll = photon_intersect(cell, r, p, v)

            if coll == nothing
                # Check other possible cells
                if vertical
                    for k ∈ 1:9  # Max cells to check
                        cy += Δy
                        cell = [cx,cy]
                        
                        coll = photon_intersect(cell, r, p, v)
                        if coll != nothing
                            break
                        end
                    end
                else
                    # Horizontal
                    for k ∈ 1:9  # Max cells to check
                        cx += Δx
                        cell = [cx,cy]
                        
                        coll = photon_intersect(cell, r, p, v)
                        if coll != nothing
                            break
                        end
                    end
                end
            end
        end

        if coll == nothing
            println("No collision found - aborting at t=$total_t")
            println("  p = $p")
            println("  v = $v")
            done = true
        else
            displacement = coll .- p
            δxy = sqrt(sum(displacement.^2))
            tmp = total_t + δxy
            if tmp < final_t
                total_t = tmp
                v = new_velocity(cell, r, v, coll)
            else
                # Reached endpoint before collision
                t = final_t - total_t
                p[1] += v[1]*t
                p[2] += v[2]*t
                println("Final point is $p")
                dist = sqrt(p[1]^2 + p[2]^2)
                println("Distance from origin = $dist")
                done = true
            end
        end
    end
end

run (generic function with 1 method)

In [5]:
run(BigFloat("10"))

Final point is BigFloat[-0.7362926986096183107757371718663917204515754910310241180155703213582650853773168, -0.669642696363571375194376070967098466862227372928775421597411093812897756001969]
Distance from origin = 0.9952629194433541608903118094267216210294669227341543498032088580730132317121767


The solution above has been verified by comparison with the solution written up by F. H. Simons.